<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/Kaggle_disaster_nlp%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install transformers

     |████████████████████████████████| 2.5 MB 15.5 MB/s 
     |████████████████████████████████| 895 kB 71.3 MB/s 
     |████████████████████████████████| 3.3 MB 55.3 MB/s 


In [29]:
pip install regex requests hydra-core omegaconf

     |████████████████████████████████| 144 kB 15.3 MB/s 
     |████████████████████████████████| 74 kB 2.1 MB/s 
     |████████████████████████████████| 112 kB 26.1 MB/s 
     |████████████████████████████████| 636 kB 23.1 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141229 sha256=4395fa1a1b66e061a3af424d07069c78b6bab8590a52bdf18c266bbd78240b52
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [64]:
import os
import math
import random
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import AdamW, AutoTokenizer, AutoModel, AutoConfig, get_cosine_schedule_with_warmup, AutoModelForMaskedLM, LineByLineTextDataset, DataCollatorForLanguageModeling, BertForSequenceClassification

from sklearn.model_selection import KFold
import gc
gc.enable()

In [40]:
train = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/commonlitreadabilityprize/commonlitreadabilityprize-train.csv')
test = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/commonlitreadabilityprize/commonlitreadabilityprize-test.csv')
submit = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/commonlitreadabilityprize/commonlitreadabilityprize-sample_submission.csv')

# 텍스트 전처리

In [42]:
# URL 제거
train['excerpt'] = train['excerpt'].apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(x)))
test['excerpt'] = test['excerpt'].apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(x)))

In [44]:
# # 제거
train['excerpt'] = train['excerpt'].apply(lambda x: re.sub(r'#', '', str(x)))
test['excerpt'] = test['excerpt'].apply(lambda x: re.sub(r'#', '', str(x)))

In [46]:
def emojicleaner(text):
  regrex_pattern = re.compile(pattern="["
  u"\U0001F600-\U0001F64F"  # emoticons
  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
  u"\U0001F680-\U0001F6FF"  # transport & map symbols
  u"\U0001F1E0-\U0001F1FF"  # flags(IOS)
  "]+", flags=re.UNICODE
  )
  return regrex_pattern.sub(r'', text)

train['excerpt'] = train['excerpt'].apply(lambda x: emojicleaner(str(x)))
test['excerpt'] = test['excerpt'].apply(lambda x: emojicleaner(str(x)))

In [48]:
# HTML tag 제거
train['excerpt'] = train['excerpt'].apply(lambda x: re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', str(x)))
test['excerpt'] = test['excerpt'].apply(lambda x: re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', str(x)))

In [53]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [54]:
# Remove Stopwords from tweets
import nltk
from nltk.corpus import stopwords  # nltk에 있는 stopwords

stop = stopwords.words('english')
def stepwordcleanser(word):
  if word in stop:
    return ""
  else:
    return word

train['excerpt'] = train['excerpt'].apply(lambda x: " ".join([stepwordcleanser(i) for i in str(x).split()]))
test['excerpt'] = test['excerpt'].apply(lambda x: " ".join([stepwordcleanser(i) for i in str(x).split()]))

# Bert 토큰화, 모델 불러오기

In [56]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [57]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

def MapTokens(tweet, labs='None'):
  global labels

  tokenids = []
  textmasks = []

  for text in tweet:
    encoded_dict = tokenizer.encode_plus(text,
                                         add_special_tokens=True,
                                         truncation='longest_first',
                                         max_length=84,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    tokenids.append(encoded_dict['input_ids'])
    textmasks.append(encoded_dict['attention_mask'])

  tokenids = torch.cat(tokenids, dim=0)
  textmasks = torch.cat(textmasks, dim=0)
  if labs != 'None':
    labels = torch.tensor(labs)

  return tokenids, textmasks, labels

In [58]:
train_tokenids, train_masks, train_labels = MapTokens(train['excerpt'].values, train['target'].values)
test_tokenids, test_masks, test_labels = MapTokens(train['excerpt'].values, train['target'].values)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [63]:
# create a TensorDataset
train_tensordataset = TensorDataset(train_tokenids, train_masks, train_labels)
test_tensordataset = TensorDataset(test_tokenids, test_masks, test_labels)

# split train, val
train_size = int(0.8 * len(train_tensordataset))
val_size = len(train_tensordataset) - train_size
train_dataset, valid_dataset = random_split(train_tensordataset, [train_size, val_size])

# Bert DataLoad
batch_size = 32
train_loader = DataLoader(train_dataset,
                          sampler=RandomSampler(train_dataset),
                          batch_size=batch_size)
val_loader = DataLoader(valid_dataset,
                        sampler=RandomSampler(train_dataset),
                        batch_size=batch_size)
test_loader = DataLoader(test_tensordataset,
                         sampler=SequentialSampler(test_tensordataset),
                         batch_size=batch_size)

In [66]:
# Bert Load pretrained classification
model = BertForSequenceClassification.from_pretrained('bert-large-uncased',
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states =False)

model.to(device)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [88]:
# optimizer
optimizer = AdamW(model.parameters(),
                  lr=6e-6,
                  eps=1e-8)

# scheduler
epochs = 4
scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(train_loader)*epochs)

In [89]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [90]:
training_stats = []
BERT_train_predictions = []
BERT_true_labels = []
# Measure the total training time for the whole run.
total_t0 = time.time()

print("Training Started")

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('...')
    
     
    t0 = time.time() # Measure how long the training epoch takes.
    total_train_loss = 0     # Reset the total loss for this epoch.
    
    model.train() # Set the mode and iterate using the dataloader
    
    for step, batch in enumerate(train_loader):
        
        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(BERT_Train_Loader), elapsed))

        # Read three pytorch tensors (input ids, attention masks, labels) in each Batch Step
        b_input_ids = batch[0].to(device).to(torch.int64)
        b_input_mask = batch[1].to(device).to(torch.int64)
        b_labels = batch[2].to(device).to(torch.int64)
        
        model.zero_grad()# Clear any previously calculated gradients        
        # Evaluate the model on this training batch
          
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
               
        total_train_loss += loss.item() # Accumulate the training loss  
        
        loss.backward()# Perform a backward pass to calculate the gradients.
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clip the norm of the gradients to 1.0 -'exploding gradients' problem.
        
        BERT_Tweet_Optimizer.step() # Move to Next Step
        
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        BERT_train_predictions.append(logits)
        BERT_true_labels.append(label_ids)
        Linear_Scheduler.step()

    # Calculate the average loss over all of the batches.
    
        # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_loader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
      
    
    print(avg_train_loss)
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
        }
    )
        
print('')
print('Training complete!')

Training Started

======== Epoch 1 / 4 ========
...


IndexError: ignored